In [1]:
from bs4 import BeautifulSoup as bs
import requests
import csv
import pandas as pd
import numpy as np

## Get List of Champions

In [2]:
r = requests.get('https://leagueoflegends.fandom.com/wiki/List_of_champions').text
soup = bs(r)

list_champions = soup.find('table', class_='article-table')

In [3]:
champ_row = list_champions.tbody.find_all('tr')
champ_info = []

for index, row in enumerate(champ_row):
    try:
        if index == 0:
            continue
        else:
            data = {}
            table_data = row.find_all('td')
            champ = table_data[0].find_all('a')[1]
            name, desc = champ.get_text('|', strip=True).split('|')
            href = champ['href']
            data['name'] = name
            data['description'] = desc
            data['href'] = href

            champ_class = table_data[1]['data-sort-value']
            data['class'] = champ_class

            release_date = table_data[2].get_text(strip=True)
            data['release_date'] = release_date

            last_changed = table_data[3].get_text(strip=True)
            data['last_changed'] = last_changed

            be_cost = table_data[4].get_text(strip=True)
            data['be_cost'] = be_cost

            rp_cost = table_data[5].get_text(strip=True)
            data['rp_cost'] = rp_cost

            champ_info.append(data)
                
    except Exception as e:
        print(f'{index} uh-oh')

In [4]:
keys = champ_info[0].keys()
with open('champions.csv', 'w') as f:
    writer = csv.DictWriter(f, keys)
    writer.writeheader()
    writer.writerows(champ_info)
    

In [5]:
df = pd.read_csv('champions.csv')
df.head()

,name,description,href,class,release_date,last_changed,be_cost,rp_cost
0,Aatrox,the Darkin Blade,/wiki/Aatrox/LoL,Juggernaut,2013-06-13,V11.19,4800,880
1,Ahri,the Nine-Tailed Fox,/wiki/Ahri/LoL,Burst,2011-12-14,V12.3,3150,790
2,Akali,the Rogue Assassin,/wiki/Akali/LoL,Assassin,2010-05-11,V11.22,3150,790
3,Akshan,the Rogue Sentinel,/wiki/Akshan/LoL,"Marksman,Assassin",2021-07-22,V12.3,6300,975
4,Alistar,the Minotaur,/wiki/Alistar/LoL,Vanguard,2009-02-21,V11.11,1350,585


In [6]:
df.shape

(158, 8)

In [7]:
r = requests.get('https://leagueoflegends.fandom.com/wiki/List_of_champions/Base_statistics').text
soup = bs(r)
stat_box = soup.find('table', class_='sortable wikitable sticky-header')

stat_table = stat_box.find('tbody')
champion = stat_table.find_all('tr')

In [8]:
keys = []

for i in champion[0]:
    keys.append(i.get_text())

In [9]:
def populate_dict(keys, value):
    data = {}
    for i, keys in enumerate(keys):
        data[keys] = value[i].get_text()
    return data

champ_stat = []
for index, c in enumerate(champion):
    if index == 0:
        continue
    stat = c.find_all('td')    
    stat_dict = populate_dict(keys, stat)
    champ_stat.append(stat_dict)

In [10]:
with open('champion_stats.csv', 'w') as f:
    writer = csv.DictWriter(f, keys)
    writer.writeheader()
    writer.writerows(champ_stat)

In [11]:
df = pd.read_csv('champion_stats.csv')
df.head()

,Champions,HP,HP+,HP5,HP5+,MP,MP+,MP5,MP5+,AD,AD+,AS,AS+,AR,AR+,MR,MR+,MS,Range
0,Aatrox,580.0,90,3.00,1.00,0.0,0.0,0.000,0.0,60.0,5.00,0.651,+2.5%,38.0,3.25,32.0,1.25,345,175
1,Ahri,500.0,82,2.50,0.60,418.0,25.0,8.000,0.8,53.0,3.00,0.668,+2%,18.0,3.50,30.0,0.50,330,550
2,Akali,500.0,105,9.00,0.90,200.0,0.0,50.000,0.0,62.0,3.30,0.625,+3.2%,23.0,3.50,37.0,1.25,345,125
3,Akshan,560.0,90,3.75,0.65,350.0,40.0,8.175,0.7,52.0,3.50,0.638,+4%,26.0,3.00,30.0,0.50,330,500
4,Alistar,600.0,106,8.50,0.85,350.0,40.0,8.500,0.8,62.0,3.75,0.625,+2.125%,44.0,3.50,32.0,1.25,330,125


In [12]:
df.shape

(160, 19)